In [ ]:
import illustris_python as il
import matplotlib.pyplot as plt
from tenet.tenet.util import sphMap
import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.patheffects as pe
import matplotlib.transforms as transforms
from matplotlib.gridspec import GridSpec
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.ndimage import gaussian_filter
from scipy.ndimage import fourier_gaussian
from scipy.stats import ks_2samp, anderson_ksamp
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
from scipy import interpolate
import os
import time
import h5py
import rohr_utils as ru 
import random
import six
%matplotlib inline 

plt.style.use('fullpage.mplstyle')
figsizewidth  = 6.902 # the textwidth in inches of MNRAS


In [ ]:
def model_2dradprof(data):
    """ 
    compute the model 2D radial profile from a 2D histogram
    based on StackOverflow answer found here: 
    https://stackoverflow.com/questions/21242011/most-efficient-way-to-calculate-radial-profile
    """
    center = [(data.shape[0] - 1.) / 2., (data.shape[1] - 1.) / 2.]
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(int)

    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    
    rpix = np.arange(radialprofile.size)
    radprof_func = interpolate.interp1d(rpix, radialprofile)
    
    model = radprof_func(r)
    
    return model, radprof_func 


In [ ]:
indirec = '../Output/L680n8192TNG_subfindGRP/'
infname = 'central_subfind_L680n8192TNG_tau.hdf5'
centrals_taudict = {}
with h5py.File(indirec + infname, 'a') as f:
    group = f['Group']
    for dset_key in group.keys():
        centrals_taudict[dset_key] = group[dset_key][:]
    f.close()
    
haloIDs = centrals_taudict['HostSubhaloGrNr']


In [ ]:
indirec = '../Output/L680n8192TNG_subfindGRP/'
infname = 'subfind_L680n8192TNG_tau_gasz0.hdf5'
satellites_taudict = {}
with h5py.File(indirec + infname, 'a') as f:
    group = f['Group']
    for dset_key in group.keys():
        satellites_taudict[dset_key] = group[dset_key][:]
    f.close()


In [ ]:
SubhaloHotGasMass_z0 = satellites_taudict['SubhaloHotGasMass_z0']
HCDnz0_key = 'HostCentricDistance_norm_z0'
HCDnz0 = satellites_taudict[HCDnz0_key]

subfindIDs = satellites_taudict['SubfindID']
HostSubhaloGrNr = satellites_taudict['HostSubhaloGrNr']

HCDnz0_key = 'HostCentricDistance_norm_z0'
HCDnz0 = satellites_taudict[HCDnz0_key]

plt.figure()
plt.hist(HCDnz0)

plt.figure()
plt.hist(np.log10(SubhaloHotGasMass_z0))

In [ ]:
# load a low resultion halo's gas cells
sim = 'L680n8192TNG'
basePath = ru.ret_basePath(sim)
snapNum = 99

# load general simulation parameters
header = il.groupcat.loadHeader(basePath, snapNum)
z = header['Redshift']
a = header['Time'] # scale factor
h = header['HubbleParam'] # = 0.6774
boxsize = header['BoxSize'] * a / h
    
gas_ptn = il.util.partTypeNum('gas')



In [ ]:
halo_index = 33
haloID = haloIDs[halo_index]
satellite_taudict_indices = np.where(haloID == HostSubhaloGrNr)[0]
subhaloIDs = subfindIDs[satellite_taudict_indices]
print(subhaloIDs)

# load the FoF gas cels
gas_fields = ['Masses', 'Coordinates', 'Density', 'GFM_Metallicity',
              'InternalEnergy', 'ElectronAbundance', 'StarFormationRate']


In [ ]:
xray_key = 'xray_lum_0.5-2.0kev'
total_gas_cells = il.snapshot.loadOriginalZoom(basePath, snapNum, haloID, gas_ptn, fields=gas_fields)
total_gas_cells[xray_key] = np.zeros(total_gas_cells['count'], dtype=float) - 1.
direc = '/vera/ptmp/gc/dnelson/sims.TNG/L680n8192TNG/data.files/cache/'
fname = 'cached_gas_xray_lum_0.5-2.0kev_99.hdf5'
with h5py.File(direc + fname, 'r') as f:
    # load fuzz length, compute offset, call loadSubset                                                                     
    subset = il.snapshot.getSnapOffsets(basePath, snapNum, haloID, "Group")
    
    # identify original halo ID and corresponding index
    halo = il.snapshot.loadSingle(basePath, snapNum, haloID=haloID)
    assert 'GroupOrigHaloID' in halo, 'Error: loadOriginalZoom() only for the TNG-Cluster simulation.'
    orig_index = np.where(subset['HaloIDs'] == halo['GroupOrigHaloID'])[0][0]

    # (1) load all FoF particles/cells
    length_FoF = subset['GroupsTotalLengthByType'][orig_index, gas_ptn]
    start_FoF = subset['GroupsSnapOffsetByType'][orig_index, gas_ptn]
    total_gas_cells[xray_key][:length_FoF] = f['xray_lum_0.5-2.0kev'][start_FoF:start_FoF+length_FoF]
    
    # (2) load all non-FoF particles/cells
    length_fuzz = subset['OuterFuzzTotalLengthByType'][orig_index, gas_ptn]
    start_fuzz = subset['OuterFuzzSnapOffsetByType'][orig_index, gas_ptn]
    total_gas_cells[xray_key][length_FoF:] = f['xray_lum_0.5-2.0kev'][start_fuzz:start_fuzz+length_fuzz]
    
    f.close()
    

In [ ]:
Masses = total_gas_cells['Masses'] * 1.0e10 / h
Coordinates = total_gas_cells['Coordinates'] * a / h
Densities = total_gas_cells['Density'] * 1.0e10 / h / (a / h)**3
Sizes = (Masses / (Densities * 4./3. * np.pi))**(1./3.)
Lx_DN = total_gas_cells[xray_key]

In [ ]:
# calculate the soft x-ray image 
# project onto 2D grid 
halo = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
R200c = halo['Group_R_Crit200'] * a / h
halo_pos = halo['GroupPos'] * a / h

# we start by ignoring the z axis and a pixel scale of 1kpc
pixel_size = 1. # kpc
smoothing_scale = 200. / pixel_size # keep constant at 200 or 500 kpc
contours_smoothing = 10. / pixel_size
axes = [0,1]
boxSizeImg = [3.*R200c, 3.*R200c] # kpc
boxSizeSim = [boxsize, boxsize, boxsize]
boxCen = halo_pos[axes]
nPixels = [int(boxSizeImg[0] / pixel_size), int(boxSizeImg[1] / pixel_size)]
ndims = 3

quant=None
FoF_Lxsoft = sphMap.sphMap(Coordinates[:,axes], Sizes, Lx_DN, quant, axes, boxSizeImg, boxSizeSim, boxCen, nPixels, ndims, colDens=True)

In [ ]:
# now plot and model 
fig, big_axs = plt.subplots(2, 3, figsize=(figsizewidth, figsizewidth/1.4))

fig.suptitle(r'TNG-Cluster $z=0$ halo %d Soft X-ray Surface Brightness [0.2-2.0 keV]'%(haloID))

cmap = mpl.cm.get_cmap('cubehelix').copy()
extent = [-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[1]/2., boxSizeImg[1]/2.]
vmax = np.percentile(FoF_Lxsoft, 99.9)
Lx_imshow_kwargs = dict(extent=extent, cmap=cmap, norm=mpl.colors.LogNorm(vmin=1.0e4, vmax=vmax), origin='lower')
Lxsoft_label = 'Soft X-ray Surface Brightness \n' + r'[$10^{30}\, {\rm erg\, s^{-1}\, kpc^{-2}}$]'
cbar_kwargs = dict(extend='both', label=Lxsoft_label)

### top row
axs = big_axs[0,:]

# plot the FoF ddata
ax = axs[0]
img = ax.imshow(FoF_Lxsoft, **Lx_imshow_kwargs)
ax.set_xticks([])
ax.set_yticks([])

# add circles at the important radii    
circ_kwargs = dict(fill=False, color='white', lw=1.0)
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
x0 = -boxSizeImg[0]/2. + (boxSizeImg[0]/2. * 0.075)# kpc
y0 = boxSizeImg[1]/2. -(boxSizeImg[1]/2. * 0.075)# kpc
length = 1.0e3 # kpc
ax.plot([x0, x0+length], [y0, y0], color='white', marker='None', ls='-')
ax.text(x0+length/2., y0-(boxSizeImg[1]/2. * 0.075), '%d Mpc'%(length/1.0e3), ha='center', va='top', fontsize='small', color='white')
ax.set_xlabel(r'All Gas', fontsize='small')

# model the data and plot the projected radial profile 
FoF_Lxsoft_fft = np.fft.fft2(FoF_Lxsoft)
FoF_Lxsoft_smooth_fft = fourier_gaussian(FoF_Lxsoft_fft, smoothing_scale)
FoF_Lxsoft_smooth = np.fft.ifft2(FoF_Lxsoft_smooth_fft).real
FoF_Lxsoft_smooth_RP, FoF_Lxsoft_smooth_RP_func = model_2dradprof(FoF_Lxsoft)

ax = axs[1]
img = ax.imshow(FoF_Lxsoft_smooth, **Lx_imshow_kwargs)
cbar = fig.colorbar(img, ax=ax, extend='both')
cbar.set_label(Lxsoft_label, fontsize='small')
ax.set_xticks([])
ax.set_yticks([])
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
ax.legend(loc='upper right', labelcolor='white', fontsize='small')
ax.set_xlabel(r'Smoothed Model [%d kpc]'%(int(smoothing_scale * pixel_size)), fontsize='small')
ax = axs[2]
x = np.logspace(np.log10(1./2.), np.log10(boxSizeImg[0]/2. / pixel_size))
ax.plot(x * pixel_size, FoF_Lxsoft_smooth_RP_func(x), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Halo-Centric Distance [kpc]', fontsize='small')
ax.axvline(R200c, color='black', ls='--', marker='None')
#ax.set_ylabel(Lxsoft_label)

### bottom row
axs = big_axs[1,:]

FoF_LXsoftexcess = (FoF_Lxsoft / FoF_Lxsoft_smooth)

vmax = np.percentile(FoF_LXsoftexcess, 99.9)

ax = axs[0]
norm = mpl.colors.LogNorm(vmin=10.**(-np.log10(vmax)), vmax=vmax)
excess_kwargs = dict(cmap='Spectral', origin='lower', norm=norm)
img = ax.imshow(FoF_LXsoftexcess, extent=extent, **excess_kwargs)
cbar = fig.colorbar(img, ax=axs[1], extend='both')
LXsoftexcess_label = 'Soft X-ray SB Excess \n [All Gas / Model]'
cbar.set_label(LXsoftexcess_label, fontsize='small')

circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', color='black', fill=False)
ax.add_patch(circ_R200c)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel(LXsoftexcess_label, fontsize='small')

subhaloPos = np.zeros((subhaloIDs.size, 3), dtype=float) 
satellite_HCDNz0 = np.zeros(subhaloIDs.size, dtype=float)
satellite_SHGMz0 = satellite_HCDNz0.copy()
for i, taudict_i in enumerate(satellite_taudict_indices):
    satellite_HCDNz0[i] = satellites_taudict[HCDnz0_key][taudict_i]
    satellite_SHGMz0[i] = satellites_taudict['SubhaloHotGasMass_z0'][taudict_i]
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloIDs[i])
    subhaloPos[i] = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)

subhaloPos_xy = subhaloPos[:,axes]
satellites_fov_indices = np.where((abs(subhaloPos_xy[:,0]) < R200c) & (abs(subhaloPos_xy[:,1]) < R200c))[0]

satellite_fov_index = satellites_fov_indices[1]
subhaloID = subhaloIDs[satellite_fov_index]
subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloID)
SubhaloPos = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)
satellite_HCDNz0 = satellites_taudict[HCDnz0_key][satellite_taudict_indices[satellite_fov_index]]

size = 3.0e2 # kpc

center = [(FoF_LXsoftexcess.shape[0] - 1.) / 2., (FoF_LXsoftexcess.shape[1] - 1.) / 2.]
extent_indices = [int(center[0] + (SubhaloPos[axes][1] - size)/pixel_size), int(center[0] + (SubhaloPos[axes][1] + size)/pixel_size),
                  int(center[1] + (SubhaloPos[axes][0] - size)/pixel_size), int(center[1] + (SubhaloPos[axes][0] + size)/pixel_size)]
extent = [-size, size, -size, size]

# draw a square around the satellite, and include the zoom lines
ax1 = axs[0]
ax2 = axs[1]
xy = (SubhaloPos[axes][0] - size, SubhaloPos[axes][1] - size)
height = size*2
width = size*2

rect = patches.Rectangle(xy, width, height, fill=False)
ax.add_patch(rect)
con = ConnectionPatch(xyA=xy, coordsA=ax1.transData,
                      xyB=(0,0), coordsB=ax2.transAxes)
fig.add_artist(con)

xy2 = (xy[0], xy[1] + height)
con = ConnectionPatch(xyA=xy2, coordsA=ax1.transData,
                      xyB=(0,1), coordsB=ax2.transAxes)
fig.add_artist(con)

ax = axs[1]
vals = FoF_LXsoftexcess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]]
img = ax.imshow(vals, extent=extent, **excess_kwargs)
#cbar = fig.colorbar(img, ax=ax, extend='both', location='bottom')
#cbar.set_label(LXsoftexcess_label, fontsize='small')
x0 = -size + (size*0.15)# kpc
y0 =  size - (size*0.1)# kpc
length = 1.0e2 # kpc
ax.plot([x0, x0+length], [y0, y0], color='black', marker='None', ls='-')
ax.text(x0+length/2., y0-(size * 0.1), '%d kpc'%(length), ha='center', va='top', fontsize='small', color='black')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('Zoom-in on an \n example satellite (%d)'%subhaloID, fontsize='small')
levels = [1., 2., 5., 10.]
colors = ['lightgray', 'darkgray', 'tab:gray', 'black']

vals_fft = np.fft.fft2(vals)
vals_smooth_fft = fourier_gaussian(vals_fft, contours_smoothing)
vals_smoothed = np.fft.ifft2(vals_smooth_fft).real
#vals_smoothed = gaussian_filter(vals, contours_smoothing)
ax.contour(vals_smoothed, levels=[1, 2., 5., 10.], colors=colors, extent=extent)

ax = axs[2]
model, func = model_2dradprof(vals)
x = np.logspace(np.log10(1./2), np.log10(size / pixel_size))
ax.plot(x * pixel_size, func(x), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Satellite-Centric Distance [kpc]', fontsize='small')
intercepts = np.interp(levels, func(np.arange(size / pixel_size))[::-1], np.arange(size / pixel_size)[::-1]) * pixel_size
ax.hlines(levels, 0, intercepts, colors=colors, ls='--')
ax.vlines(intercepts, 0, levels, colors=colors, ls='--')

savefig=False
direc = '../Figures/TNGCluster/Maps/Lxsoft_multipanel/'
if savefig:
    fname = '%s_snap099_haloID%08d_LxAPEC_model_subahlo%08d_multipanel_ALL.pdf'%(sim, haloID, subhaloID)
    fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:
subhaloPos = np.zeros((subhaloIDs.size, 3), dtype=float) 
satellite_HCDNz0 = np.zeros(subhaloIDs.size, dtype=float)
satellite_SHGMz0 = satellite_HCDNz0.copy()
for i, taudict_i in enumerate(satellite_taudict_indices):
    satellite_HCDNz0[i] = satellites_taudict[HCDnz0_key][taudict_i]
    satellite_SHGMz0[i] = satellites_taudict['SubhaloHotGasMass_z0'][taudict_i]
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloIDs[i])
    subhaloPos[i] = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)

subhaloPos_axes = subhaloPos[:,axes]
satellites_fov_indices = np.where((abs(subhaloPos_axes[:,0]) < R200c*2.0) & (abs(subhaloPos_axes[:,1]) < R200c*2.0))[0]

print(satellite_taudict_indices.size, satellites_fov_indices.size)

size = 3.0e2 # kpc
x = np.logspace(np.log10(1./2), np.log10(size / pixel_size))
xs = np.tile(x, (satellites_fov_indices.size,1)) * pixel_size
ys = np.zeros((satellites_fov_indices.size, x.size), dtype=x.dtype) - 1
for i, satellite_fov_index in enumerate(satellites_fov_indices):
    subhaloID = subhaloIDs[satellite_fov_index]
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloID)
    SubhaloPos = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)
    satellite_HCDNz0 = satellites_taudict[HCDnz0_key][satellite_taudict_indices[satellite_fov_index]]

    size = 3.0e2 # kpc

    center = [(FoF_LXsoftexcess.shape[0] - 1.) / 2., (FoF_LXsoftexcess.shape[1] - 1.) / 2.]
    extent_indices = [int(center[0] + (SubhaloPos[axes][1] - size)/pixel_size), int(center[0] + (SubhaloPos[axes][1] + size)/pixel_size),
                      int(center[1] + (SubhaloPos[axes][0] - size)/pixel_size), int(center[1] + (SubhaloPos[axes][0] + size)/pixel_size)]
    if min(extent_indices) < 0:
        print('satellite on the edge of the FoV: i %d'%i)
        continue
    vals = FoF_LXsoftexcess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]]
    model, func = model_2dradprof(vals)
    ys[i,:] = func(x)
    
ys = np.ma.masked_values(ys, -1)

In [ ]:
fig, ax = plt.subplots()
ys_median = np.ma.median(ys, axis=0)
ys_mean = np.ma.mean(ys, axis=0)
ax.plot(xs.T, ys.T, 'k-', lw=1)
ax.plot(x * pixel_size, ys_median, 'b--', lw=4)
ax.plot(x * pixel_size, ys_mean, 'r--', lw=4)
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
# now plot and model 
fig, axs = plt.subplots(1, 3, figsize=(figsizewidth, figsizewidth/2.7))

fig.suptitle(r'TNG-Cluster $z=0$ halo %d Soft X-ray Surface Brightness (APEC)'%(haloID))

cmap = mpl.cm.get_cmap('cubehelix').copy()
masked_array = np.ma.masked_where(FoF_Lxsoft == 0, FoF_Lxsoft)
cmap.set_bad('black')
Lx_imshow_kwargs = dict(extent=[-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[1]/2., boxSizeImg[1]/2.], cmap=cmap, norm=mpl.colors.LogNorm(vmax=1.0e9, vmin=1.0e4), origin='lower')
Lxsoft_label = 'Soft X-ray Surface Brightness \n' + r'$\Sigma_{\rm 0.2-2\, keV} / 10^{30}\, {\rm erg\, s^{-1}\, kpc^{-2}}$'
cbar_kwargs = dict(extend='both', label=Lxsoft_label)

# plot the FoF ddata
ax = axs[0]
img = ax.imshow(masked_array, **Lx_imshow_kwargs)
ax.set_xticks([])
ax.set_yticks([])

# add circles at the important radii    
circ_kwargs = dict(fill=False, color='white', lw=1.0)
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
x0 = -boxSizeImg[0]/2. + (boxSizeImg[0]/2. * 0.075)# kpc
y0 = boxSizeImg[1]/2. -(boxSizeImg[1]/2. * 0.075)# kpc
length = 1.0e3 # kpc
ax.plot([x0, x0+length], [y0, y0], color='white', marker='None', ls='-')
ax.text(x0+length/2., y0-(boxSizeImg[1]/2. * 0.075), '%d Mpc'%(length/1.0e3), ha='center', va='top', fontsize='small', color='white')
ax.set_xlabel(r'All FoF gas cells', fontsize='small')

# model the data and plot the projected radial profile 
FoF_Lxsoft_smooth = gaussian_filter(FoF_Lxsoft, smoothing_scale)
FoF_Lxsoft_smooth_RP, FoF_Lxsoft_smooth_RP_func = model_2dradprof(FoF_Lxsoft)
#FoF_Lxsoft_model, FoF_Lxsoft_model_func = model_2dradprof(FoF_Lxsoft)

ax = axs[1]
img = ax.imshow(FoF_Lxsoft_smooth, **Lx_imshow_kwargs)
cbar = fig.colorbar(img, ax=ax, extend='both')
cbar.set_label(Lxsoft_label, fontsize='small')
ax.set_xticks([])
ax.set_yticks([])
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
ax.legend(loc='upper right', labelcolor='white', fontsize='small')
ax.set_xlabel(r'Smoothed Model', fontsize='small')
ax = axs[2]
x = np.arange(boxSizeImg[0]/2. / pixel_size)
ax.plot(x * pixel_size, FoF_Lxsoft_smooth_RP_func(x), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Halo-Centric Distance $[R / {\rm kpc}]$', fontsize='small')
ax.axvline(R200c, color='black', ls='--', marker='None')
#ax.set_ylabel(Lxsoft_label)

savefig=False
direc = '../Figures/TNGCluster/Maps/Lxsoft_multipanel/'
if savefig:
    fname = '%s_snap099_haloID%08d_LxAPEC_model_multipanel.pdf'%(sim, haloID)
    fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:
# now plot and model 
fig, axs = plt.subplots(1, 3, figsize=(figsizewidth, figsizewidth/2.7))

fig.suptitle(r'TNG-Cluster $z=0$ halo %d Soft X-ray Surface Brightness (APEC)'%(haloID))

cmap = mpl.cm.get_cmap('cubehelix').copy()
masked_array = np.ma.masked_where(FoF_Lxsoft == 0, FoF_Lxsoft)
cmap.set_bad('black')
Lx_imshow_kwargs = dict(extent=[-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[1]/2., boxSizeImg[1]/2.], cmap=cmap, norm=mpl.colors.LogNorm(vmax=1.0e9, vmin=1.0e4), origin='lower')
Lxsoft_label = 'Soft X-ray Surface Brightness \n' + r'$\Sigma_{\rm 0.2-2\, keV} / 10^{30}\, {\rm erg\, s^{-1}\, kpc^{-2}}$'
cbar_kwargs = dict(extend='both', label=Lxsoft_label)

# plot the FoF ddata
ax = axs[0]
img = ax.imshow(masked_array, **Lx_imshow_kwargs)
ax.set_xticks([])
ax.set_yticks([])

# add circles at the important radii    
circ_kwargs = dict(fill=False, color='white', lw=1.0)
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
x0 = -boxSizeImg[0]/2. + (boxSizeImg[0]/2. * 0.075)# kpc
y0 = boxSizeImg[1]/2. -(boxSizeImg[1]/2. * 0.075)# kpc
length = 1.0e3 # kpc
ax.plot([x0, x0+length], [y0, y0], color='white', marker='None', ls='-')
ax.text(x0+length/2., y0-(boxSizeImg[1]/2. * 0.075), '%d Mpc'%(length/1.0e3), ha='center', va='top', fontsize='small', color='white')
ax.set_xlabel(r'All FoF gas cells', fontsize='small')

# model the data and plot the projected radial profile 
FoF_Lxsoft_model, FoF_Lxsoft_model_func = model_2dradprof(FoF_Lxsoft)

ax = axs[1]
img = ax.imshow(FoF_Lxsoft_model, **Lx_imshow_kwargs)
cbar = fig.colorbar(img, ax=ax, extend='both')
cbar.set_label(Lxsoft_label, fontsize='small')
ax.set_xticks([])
ax.set_yticks([])
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
ax.legend(loc='upper right', labelcolor='white', fontsize='small')
ax.set_xlabel(r'Projected Radial Profile', fontsize='small')
ax = axs[2]
x = np.arange(boxSizeImg[0]/2. / 100.)
ax.plot(x, FoF_Lxsoft_model_func(x), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Halo-Centric Distance $[R / {\rm kpc}]$', fontsize='small')
ax.axvline(R200c, color='black', ls='--', marker='None')
#ax.set_ylabel(Lxsoft_label)

savefig=False
direc = '../Figures/TNGCluster/Maps/Lxsoft_multipanel/'
if savefig:
    fname = '%s_snap099_haloID%08d_LxAPEC_model_multipanel.pdf'%(sim, haloID)
    fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:
# load the FoF gas cels
gas_fields = ['Masses', 'Coordinates', 'Density', 'GFM_Metallicity',
              'InternalEnergy', 'ElectronAbundance', 'StarFormationRate']

halo = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
halo_gas_cells = il.snapshot.loadHalo(basePath, snapNum, haloID, gas_ptn, fields=gas_fields)
halo_gas_cells = ru.calc_temp_dict(halo_gas_cells)

# we only care about hot gas cells for now, eventually x-ray emitting gas cells
indices = halo_gas_cells['Temperature'] > 10.**(4.5)
for key in halo_gas_cells:
    if key == 'count':
        halo_gas_cells[key] = indices[indices].size
    else:
        halo_gas_cells[key] = halo_gas_cells[key][indices]
        
Masses = halo_gas_cells['Masses'] * 1.0e10 / h
Coordinates = halo_gas_cells['Coordinates'] * a / h
Densities = halo_gas_cells['Density'] * 1.0e10 / h / (a / h)**3
Sizes = (Masses / (Densities * 4./3. * np.pi))**(1./3.)


In [ ]:
import numpy as np
from astropy import units as u
import sys
h_const = 0.673 # Hubble constant
k_B_cgs = 1.38064852e-16 # boltzmann constant (CGS)
m_p_cgs = 1.6726219e-24 # proton mass [gram]
X_H = 0.76 # hydrogen mass fraction
gamma_gas = 5.0/3


def calculate_Xray_BolometricLum_Arepo(data):
    """ This function (Written by D. Nelson with minor modifications by M. Ayromlou) computes X-ray luminosity using Navarro+ (1995) Eqn. 6, the most basic estimator of bolometric X-ray luminosity in [10^30 erg/s] for individual gas cells, based only on their density and temperature. It assumes simplified (primordial) high-temp cooling function, and only free-free (bremsstrahlung) emission contribution from T>10^6 Kelvin gas. All inputs in code (Arepo) units. """

    # calculate mean molecular weight
    mu = calculate_mean_molecular_weight_Arepo(data).astype('float64')

    # calculate temperature (keV)
    temp = calculate_temperature_Arepo(data,unit='Kelvin')
    temp_Kelvin = temp * u.Kelvin
    temp_keV = temp_Kelvin.to(u.keV, equivalencies=u.temperature_energy()) / u.keV 

    # Eqn. 6
    mass_cgs = convert_units_Arepo(data['Masses'],quantity='mass',MAorCGS='CGS')
    rho_cgs = convert_units_Arepo(data['Density'],quantity='density',MAorCGS='CGS')

    L_x = (1.2e-24/ (mu**2)) * mass_cgs *  rho_cgs * np.sqrt(temp_keV)

    # clip any cells on eEOS (SFR>0) to zero
    dummy = np.where(data['StarFormationRate'] > 0.0)
    L_x[dummy] = 0.0

    # implement a linear ramp from log(T)=6.0 to log(T)=5.8 over which we clip to zero. This step is necessary to avoid having a non-zero X-ray luminosity for gas with T<10^6 K, which is not expected from Navarro+95 Eqn. 6.
    temp = np.log10(temp)
    L_x *= np.clip( (temp-5.8)/0.2, 0.0, 1.0 )

    L_x = np.array(L_x/1e30,dtype=np.float32) # work in this unit system of [10^30 erg/s] for xray to avoid overflows to inf

    return (L_x)


def calculate_mean_molecular_weight_Arepo(data):
    # mean molecular weight of the gas: see TNG FAQ webpage for details
    e_abund = data['ElectronAbundance']
    mu = 4*m_p_cgs/(1 + 3*X_H + 4*X_H*e_abund) # mean molecular weight
    return(mu)

def calculate_temperature_Arepo(data,unit='Kelvin'):
    # Temperature of the gas: see TNG FAQ webpage for details
    i_energy = data['InternalEnergy']
    mu = calculate_mean_molecular_weight_Arepo(data)
    gas_temp = (gamma_gas-1)*(i_energy/k_B_cgs)*mu*(10**10) # kelvin
    if unit != 'Kelvin':
        gas_temp = gas_temp * u.Kelvin
        if unit == 'keV':
            gas_temp = gas_temp.to(u.keV, equivalencies=u.temperature_energy()) / u.keV 
        elif unit == 'eV':
            gas_temp = gas_temp.to(u.eV, equivalencies=u.temperature_energy()) / u.eV
    return(gas_temp)


def convert_units_Arepo(data, quantity, tofromcode='fromcode', MAorCGS='MA'):
    """ This function converts units from Arepo units to my prefered units 
    * data: data
    * quantity [Arepo unit][my preferred unit]: mass [1e10Msun/h][Msun], scale[kpc/h][Mpc], density, density2d
    * tofromcode: tocode (convert from my units to code units), fromcode (convert from code units to my units)
    """
    
    if MAorCGS == 'MA':
        mass_conversion = 1e10/h_const
        scale_conversion = 1/(1000*h_const)
    elif MAorCGS == 'CGS':
        mass_conversion = (1e10/h_const)*1.989e33 
        scale_conversion = (3.086e24)/(1000*h_const)
    
    if quantity == 'mass':
        if tofromcode == 'fromcode':
            data = data*mass_conversion
        elif tofromcode == 'tocode':
            data = data/mass_conversion
    elif quantity == 'scale':
        if tofromcode == 'fromcode':
            data = data*scale_conversion
        elif tofromcode == 'tocode':
            data = data/scale_conversion
    elif quantity == 'density':
        if tofromcode == 'fromcode':
            data = data*mass_conversion/(scale_conversion**3)
        elif tofromcode == 'tocode':
            data = data*(scale_conversion**3)/mass_conversion
    elif quantity == 'density2d':
        if tofromcode == 'fromcode':
            data = data*mass_conversion/(scale_conversion**2)
        elif tofromcode == 'tocode':
            data = data*(scale_conversion**2)/mass_conversion
    else:
        print("Quantity not recognized")
        sys.exit()
            
    return(data)

In [ ]:
gas_fields = ['Masses', 'Coordinates', 'Density', 'GFM_Metallicity',
              'InternalEnergy', 'ElectronAbundance', 'StarFormationRate']

halo = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
halo_gas_cells = il.snapshot.loadHalo(basePath, snapNum, haloID, gas_ptn, fields=gas_fields)
halo_gas_cells = ru.calc_temp_dict(halo_gas_cells)

In [ ]:
Lx_MA = calculate_Xray_BolometricLum_Arepo(halo_gas_cells)

In [ ]:
indices = Lx_MA > 0
plt.figure()
plt.hist(np.log10(Lx_MA[indices]) + 30)
plt.yscale('log')


In [ ]:
subset = il.snapshot.getSnapOffsets(basePath, snapNum, haloID, "Group")
start = subset['offsetType'][gas_ptn]
end = start +  subset['lenType'][gas_ptn]



In [ ]:
subset = il.snapshot.getSnapOffsets(basePath, snapNum, haloID, "Group")
start = subset['offsetType'][gas_ptn]
end = start +  subset['lenType'][gas_ptn]

direc = '/vera/ptmp/gc/dnelson/sims.TNG/L680n8192TNG/data.files/cache/'
fname = 'cached_gas_xray_lum_0.5-2.0kev_99.hdf5'
with h5py.File(direc + fname, 'r') as f:
    Lx_DN = f['xray_lum_0.5-2.0kev'][start:end]
    f.close()

In [ ]:
indices = Lx_DN > 0
plt.figure()
plt.hist(np.log10(Lx_DN[indices]) + 30)
plt.yscale('log')

In [ ]:
indices = Lx_MA > 0
y = Lx_MA[indices] / Lx_DN[indices]
x = Lx_DN[indices]
T = halo_gas_cells['Temperature'][indices]
rho = halo_gas_cells['Density'][indices] * 1.0e10 / h / (a / h)**3
Z = halo_gas_cells['GFM_Metallicity'][indices] / 0.0127

direc = '../Figures/TNGCluster/'
savefig=False

fig, axs = plt.subplots(2, 2, figsize=(figsizewidth, figsizewidth / 1.25))

xdsets = [np.log10(x) + 30, np.log10(T), np.log10(rho), np.log10(Z)]
xlabels = [r'$\log_{10}\, [L_{\rm 0.2-2\, keV}({\rm APEC}) / {\rm erg\, s^{-1}}]$',
           r'$\log_{10}\, [T_{\rm gas} / {\rm K}]$',
           r'$\log_{10}\, [\rho_{\rm gas} / {\rm M_\odot\, kpc^{-3}}]$',
           r'$\log_{10}\, [Z_{\rm gas} / {\rm Z_\odot}]$']

fig.suptitle(r'TNG-Cluster snap 099 halo %d gas ($%s$ cells)'%(haloID, ru.latex_float(x.size, 1)))
fig.supylabel(r'$\log_{10}\, [L_{\rm x}({\rm Navarro+95}) /L_{\rm 0.2-2\, keV}({\rm APEC})]$')

one_one_kwargs = dict(ls='-', marker='None', c='k', zorder=1.0, lw=2.5)

norm = mpl.colors.LogNorm(vmax=1.0e5)
bins = 500
Ncells_label = r'Number of Gas Cells'
for i, ax in enumerate(axs.flatten()):
    dset = xdsets[i]
    xlabel = xlabels[i]
    ax.set_xlabel(xlabel)

    vals = ax.hist2d(dset, np.log10(y), bins=bins, norm=norm, rasterized=True)
    ax.plot(ax.get_xlim(), [0, 0], **one_one_kwargs)

    
    cax = inset_axes(ax, width='50%', height='5%', loc='lower left')
    cbar = plt.colorbar(vals[3], cax=cax, orientation='horizontal')
    cbar.set_label(Ncells_label, labelpad=5, fontsize='small')
    cbar.ax.tick_params(labelsize='x-small')
    cax.xaxis.set_label_position('top')
    cax.xaxis.set_ticks_position('top')

if savefig:
    fname = '%s_snap099_haloID%08d_LxNavarro+95-LxAPEC_multipanel.pdf'%(sim, haloID)
    fig.savefig(direc + fname, bbox_inches='tight')

    

In [ ]:
halo_gas_cells['count'], end

In [ ]:
Coordinates = Coordinates[:,axes]
Temperatures = np.log10(halo_gas_cells['Temperature'])
quant = Temperatures
FoF_Tempmap = sphMap.sphMap(Coordinates, Sizes, Masses, quant, axes, boxSizeImg, boxSizeSim, boxCen, nPixels, ndims, colDens=False)

cmap = mpl.cm.get_cmap('turbo').copy()
masked_array = np.ma.masked_where(FoF_Tempmap == 0, FoF_Tempmap)
cmap.set_bad('white')
plt.figure()
img = plt.imshow(masked_array, extent=[-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[1]/2., boxSizeImg[1]/2.], cmap=cmap, vmin=np.percentile(FoF_Tempmap, 99)-1.0, vmax=np.percentile(FoF_Tempmap, 99), origin='lower')
cbar = plt.colorbar(img, extend='both', label=r'Mass Weighted Gas Temperature')

In [ ]:
# calculate the soft x-ray image 
Coordinates = Coordinates[:,axes]
quant=None
Lx_DN_ergs = Lx_DN
nPixels = int(boxSizeImg / 100)
FoF_Lxsoft = sphMap.sphMap(Coordinates, Sizes, Lx_DN_ergs, quant, axes, boxSizeImg, boxSizeSim, boxCen, nPixels, ndims, colDens=True)

In [ ]:
# now plot and model 
fig, axs = plt.subplots(1, 3, figsize=(figsizewidth, figsizewidth/2.7))

fig.suptitle(r'TNG-Cluster $z=0$ halo %d Soft X-ray Surface Brightness (APEC)'%(haloID))

cmap = mpl.cm.get_cmap('cubehelix').copy()
masked_array = np.ma.masked_where(FoF_Lxsoft == 0, FoF_Lxsoft)
cmap.set_bad('black')
Lx_imshow_kwargs = dict(extent=[-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[1]/2., boxSizeImg[1]/2.], cmap=cmap, norm=mpl.colors.LogNorm(vmax=1.0e9, vmin=1.0e4), origin='lower')
Lxsoft_label = 'Soft X-ray Surface Brightness \n' + r'$\Sigma_{\rm 0.2-2\, keV} / 10^{30}\, {\rm erg\, s^{-1}\, kpc^{-2}}$'
cbar_kwargs = dict(extend='both', label=Lxsoft_label)

# plot the FoF ddata
ax = axs[0]
img = ax.imshow(masked_array, **Lx_imshow_kwargs)
ax.set_xticks([])
ax.set_yticks([])

# add circles at the important radii    
circ_kwargs = dict(fill=False, color='white', lw=1.0)
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
x0 = -boxSizeImg[0]/2. + (boxSizeImg[0]/2. * 0.075)# kpc
y0 = boxSizeImg[1]/2. -(boxSizeImg[1]/2. * 0.075)# kpc
length = 1.0e3 # kpc
ax.plot([x0, x0+length], [y0, y0], color='white', marker='None', ls='-')
ax.text(x0+length/2., y0-(boxSizeImg[1]/2. * 0.075), '%d Mpc'%(length/1.0e3), ha='center', va='top', fontsize='small', color='white')
ax.set_xlabel(r'All FoF gas cells', fontsize='small')

# model the data and plot the projected radial profile 
FoF_Lxsoft_model, FoF_Lxsoft_model_func = model_2dradprof(FoF_Lxsoft)

ax = axs[1]
img = ax.imshow(FoF_Lxsoft_model, **Lx_imshow_kwargs)
cbar = fig.colorbar(img, ax=ax, extend='both')
cbar.set_label(Lxsoft_label, fontsize='small')
ax.set_xticks([])
ax.set_yticks([])
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
ax.legend(loc='upper right', labelcolor='white', fontsize='small')
ax.set_xlabel(r'Projected Radial Profile', fontsize='small')
ax = axs[2]
x = np.arange(boxSizeImg[0]/2.)
ax.plot(x, FoF_Lxsoft_model_func(x), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Halo-Centric Distance $[R / {\rm kpc}]$', fontsize='small')
ax.axvline(R200c, color='black', ls='--', marker='None')
#ax.set_ylabel(Lxsoft_label)

savefig=False
direc = '../Figures/TNGCluster/Maps/Lxsoft_multipanel/'
if savefig:
    fname = '%s_snap099_haloID%08d_LxAPEC_model_multipanel.pdf'%(sim, haloID)
    fig.savefig(direc + fname, bbox_inches='tight')



In [ ]:
from scipy.ndimage import gaussian_filter

# now plot and model 
fig, axs = plt.subplots(1, 3, figsize=(figsizewidth, figsizewidth/2.7))

fig.suptitle(r'TNG-Cluster $z=0$ halo %d Soft X-ray Surface Brightness (APEC)'%(haloID))

cmap = mpl.cm.get_cmap('cubehelix').copy()
masked_array = np.ma.masked_where(FoF_Lxsoft == 0, FoF_Lxsoft)
cmap.set_bad('black')
Lx_imshow_kwargs = dict(extent=[-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[1]/2., boxSizeImg[1]/2.], cmap=cmap, norm=mpl.colors.LogNorm(vmax=1.0e9, vmin=1.0e4), origin='lower')
Lxsoft_label = 'Soft X-ray Surface Brightness \n' + r'$\Sigma_{\rm 0.2-2\, keV} / 10^{30}\, {\rm erg\, s^{-1}\, kpc^{-2}}$'
cbar_kwargs = dict(extend='both', label=Lxsoft_label)

# plot the FoF ddata
ax = axs[0]
img = ax.imshow(masked_array, **Lx_imshow_kwargs)
ax.set_xticks([])
ax.set_yticks([])

# add circles at the important radii    
circ_kwargs = dict(fill=False, color='white', lw=1.0)
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
x0 = -boxSizeImg[0]/2. + (boxSizeImg[0]/2. * 0.075)# kpc
y0 = boxSizeImg[1]/2. -(boxSizeImg[1]/2. * 0.075)# kpc
length = 1.0e3 # kpc
ax.plot([x0, x0+length], [y0, y0], color='white', marker='None', ls='-')
ax.text(x0+length/2., y0-(boxSizeImg[1]/2. * 0.075), '%d Mpc'%(length/1.0e3), ha='center', va='top', fontsize='small', color='white')
ax.set_xlabel(r'All FoF gas cells', fontsize='small')

# model the data and plot the projected radial profile 
FoF_Lxsoft_smooth = gaussian_filter(FoF_Lxsoft, 500)
FoF_Lxsoft_smooth_RP, FoF_Lxsoft_smooth_RP_func = model_2dradprof(FoF_Lxsoft)
#FoF_Lxsoft_model, FoF_Lxsoft_model_func = model_2dradprof(FoF_Lxsoft)

ax = axs[1]
img = ax.imshow(FoF_Lxsoft_smooth, **Lx_imshow_kwargs)
cbar = fig.colorbar(img, ax=ax, extend='both')
cbar.set_label(Lxsoft_label, fontsize='small')
ax.set_xticks([])
ax.set_yticks([])
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
ax.legend(loc='upper right', labelcolor='white', fontsize='small')
ax.set_xlabel(r'Projected Radial Profile', fontsize='small')
ax = axs[2]
x = np.arange(boxSizeImg[0]/2.)
ax.plot(x, FoF_Lxsoft_smooth_RP_func(x), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Halo-Centric Distance $[R / {\rm kpc}]$', fontsize='small')
ax.axvline(R200c, color='black', ls='--', marker='None')
#ax.set_ylabel(Lxsoft_label)

savefig=False
direc = '../Figures/TNGCluster/Maps/Lxsoft_multipanel/'
if savefig:
    fname = '%s_snap099_haloID%08d_LxAPEC_model_multipanel.pdf'%(sim, haloID)
    fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:
# now plot and model 
fig, big_axs = plt.subplots(2, 3, figsize=(figsizewidth, figsizewidth/1.4))

fig.suptitle(r'TNG-Cluster $z=0$ halo %d Soft X-ray Surface Brightness (APEC)'%(haloID))

cmap = mpl.cm.get_cmap('cubehelix').copy()
masked_array = np.ma.masked_where(FoF_Lxsoft == 0, FoF_Lxsoft)
cmap.set_bad('black')
Lx_imshow_kwargs = dict(extent=[-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[1]/2., boxSizeImg[1]/2.], cmap=cmap, norm=mpl.colors.LogNorm(vmax=1.0e9, vmin=1.0e4), origin='lower')
Lxsoft_label = 'Soft X-ray Surface Brightness \n' + r'[$10^{30}\, {\rm erg\, s^{-1}\, kpc^{-2}}$]'
cbar_kwargs = dict(extend='both', label=Lxsoft_label)

### top row
axs = big_axs[0,:]

# plot the FoF ddata
ax = axs[0]
img = ax.imshow(masked_array, **Lx_imshow_kwargs)
ax.set_xticks([])
ax.set_yticks([])

# add circles at the important radii    
circ_kwargs = dict(fill=False, color='white', lw=1.0)
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
x0 = -boxSizeImg[0]/2. + (boxSizeImg[0]/2. * 0.075)# kpc
y0 = boxSizeImg[1]/2. -(boxSizeImg[1]/2. * 0.075)# kpc
length = 1.0e3 # kpc
ax.plot([x0, x0+length], [y0, y0], color='white', marker='None', ls='-')
ax.text(x0+length/2., y0-(boxSizeImg[1]/2. * 0.075), '%d Mpc'%(length/1.0e3), ha='center', va='top', fontsize='small', color='white')
ax.set_xlabel(r'All Halo Gas', fontsize='small')

# model the data and plot the projected radial profile 
FoF_Lxsoft_model, FoF_Lxsoft_model_func = model_2dradprof(FoF_Lxsoft)

ax = axs[1]
img = ax.imshow(FoF_Lxsoft_model, **Lx_imshow_kwargs)
cbar = fig.colorbar(img, ax=ax, extend='both')
cbar.set_label(Lxsoft_label, fontsize='small')
ax.set_xticks([])
ax.set_yticks([])
circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', **circ_kwargs,
                            label=r'$R_{\rm 200c} = %#1.1f\, {\rm Mpc}$'%float(R200c / 1.0e3))
ax.add_patch(circ_R200c)
ax.legend(loc='upper right', labelcolor='white', fontsize='small')
ax.set_xlabel(r'Model Radial Profile', fontsize='small')
ax = axs[2]
x = np.arange(boxSizeImg[0]/2.)
ax.plot(x, FoF_Lxsoft_model_func(x), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Halo-Centric Distance [kpc]', fontsize='small')
ax.axvline(R200c, color='black', ls='--', marker='None')
#ax.set_ylabel(Lxsoft_label)

### bottom row
axs = big_axs[1,:]

FoF_LXsoftexcess = (FoF_Lxsoft - FoF_Lxsoft_model)

ax = axs[0]
img = ax.imshow(FoF_LXsoftexcess, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.], cmap='Spectral', origin='lower',
                 norm=mpl.colors.SymLogNorm(1.0e5, vmin=-1.0e9, vmax=1.0e9))
cbar = fig.colorbar(img, ax=axs[1], extend='both')
LXsoftexcess_label = 'Soft X-ray SB Excess \n' +  r'[$10^{30}\, {\rm erg\, s^{-1}\, kpc^{-2}}$]'
cbar.set_label(LXsoftexcess_label, fontsize='small')

circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', color='black', fill=False)
ax.add_patch(circ_R200c)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('Soft X-ray SB Excess \n [All Halo Gas - Model]', fontsize='small')

subhaloPos = np.zeros((subhaloIDs.size, 3), dtype=float) 
satellite_HCDNz0 = np.zeros(subhaloIDs.size, dtype=float)
satellite_SHGMz0 = satellite_HCDNz0.copy()
for i, taudict_i in enumerate(satellite_taudict_indices):
    satellite_HCDNz0[i] = satellites_taudict[HCDnz0_key][taudict_i]
    satellite_SHGMz0[i] = satellites_taudict['SubhaloHotGasMass_z0'][taudict_i]
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloIDs[i])
    subhaloPos[i] = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)

subhaloPos_xy = subhaloPos[:,axes]
satellites_fov_indices = np.where((abs(subhaloPos_xy[:,0]) < R200c) & (abs(subhaloPos_xy[:,1]) < R200c))[0]

satellite_fov_index = satellites_fov_indices[1]
subhaloID = subhaloIDs[satellite_fov_index]
subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloID)
SubhaloPos = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)
satellite_HCDNz0 = satellites_taudict[HCDnz0_key][satellite_taudict_indices[satellite_fov_index]]

size = 3.0e2 # kpc

center = [(FoF_LXsoftexcess.shape[0] - 1.) / 2., (FoF_LXsoftexcess.shape[1] - 1.) / 2.]
extent_indices = [int(center[0] + SubhaloPos[axes][1] - size), int(center[0] + SubhaloPos[axes][1] + size),
                  int(center[1] + SubhaloPos[axes][0] - size), int(center[1] + SubhaloPos[axes][0] + size)]
extent = [-size, size, -size, size]

# draw a square around the satellite, and include the zoom lines
ax1 = axs[0]
ax2 = axs[1]
xy = (SubhaloPos[axes][0] - size, SubhaloPos[axes][1] - size)
height = size*2
width = size*2

rect = patches.Rectangle(xy, width, height, fill=False)
ax.add_patch(rect)
con = ConnectionPatch(xyA=xy, coordsA=ax1.transData,
                      xyB=(0,0), coordsB=ax2.transAxes)
fig.add_artist(con)

xy2 = (xy[0], xy[1] + height)
con = ConnectionPatch(xyA=xy2, coordsA=ax1.transData,
                      xyB=(0,1), coordsB=ax2.transAxes)
fig.add_artist(con)

ax = axs[1]
img = ax.imshow(FoF_LXsoftexcess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]], extent=extent, cmap='GnBu', origin='lower',
                norm=mpl.colors.LogNorm(1.0e5, vmax=1.0e9))
#cbar = fig.colorbar(img, ax=ax, extend='both', location='bottom')
#cbar.set_label(LXsoftexcess_label, fontsize='small')
x0 = -size + (size*0.15)# kpc
y0 =  size - (size*0.1)# kpc
length = 1.0e2 # kpc
ax.plot([x0, x0+length], [y0, y0], color='black', marker='None', ls='-')
ax.text(x0+length/2., y0-(size * 0.1), '%d kpc'%(length), ha='center', va='top', fontsize='small', color='black')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('Zoom-in on an \n example satellite (%d)'%subhaloID, fontsize='small')


ax = axs[2]
model, func = model_2dradprof(FoF_LXsoftexcess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]])
ax.plot(np.arange(size), func(np.arange(size)), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Satellite-Centric Distance [kpc]', fontsize='small')
#ax.set_ylabel(LXsoftexcess_label)

savefig=False
direc = '../Figures/TNGCluster/Maps/Lxsoft_multipanel/'
if savefig:
    fname = '%s_snap099_haloID%08d_LxAPEC_model_subahlo%08d_multipanel.pdf'%(sim, haloID, subhaloID)
    fig.savefig(direc + fname, bbox_inches='tight')


In [ ]:
FoF_LXsoftexcess = (FoF_Lxsoft / FoF_Lxsoft_model)
fig, axs = plt.subplots(1, 3, figsize=(figsizewidth,figsizewidth/2.5))

ax = axs[0]
img = ax.imshow(FoF_LXsoftexcess, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.], cmap='Spectral', origin='lower',
                 norm=mpl.colors.LogNorm(vmin=1.0e-2, vmax=1.0e2))
cbar = fig.colorbar(img, ax=ax, extend='both', location='bottom')
LXsoftexcess_label = 'Soft X-ray Excess \n' +  r'[$10^{30}\, {\rm erg\, s^{-1}\, kpc^{-2}}$]'
cbar.set_label(LXsoftexcess_label, fontsize='small')

circ_R200c = patches.Circle((0.0,0.0), R200c, linestyle='-', color='black', fill=False)
ax.add_patch(circ_R200c)
ax.set_xticks([])
ax.set_yticks([])

subhaloPos = np.zeros((subhaloIDs.size, 3), dtype=float) 
satellite_HCDNz0 = np.zeros(subhaloIDs.size, dtype=float)
satellite_SHGMz0 = satellite_HCDNz0.copy()
for i, taudict_i in enumerate(satellite_taudict_indices):
    satellite_HCDNz0[i] = satellites_taudict[HCDnz0_key][taudict_i]
    satellite_SHGMz0[i] = satellites_taudict['SubhaloHotGasMass_z0'][taudict_i]
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloIDs[i])
    subhaloPos[i] = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)

subhaloPos_xy = subhaloPos[:,axes]
satellites_fov_indices = np.where((abs(subhaloPos_xy[:,0]) < R200c) & (abs(subhaloPos_xy[:,1]) < R200c))[0]

satellite_fov_index = satellites_fov_indices[1]
subhaloID = subhaloIDs[satellite_fov_index]
subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloID)
SubhaloPos = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)
satellite_HCDNz0 = satellites_taudict[HCDnz0_key][satellite_taudict_indices[satellite_fov_index]]

size = 3.0e2 # kpc

center = [(FoF_LXsoftexcess.shape[0] - 1.) / 2., (FoF_LXsoftexcess.shape[1] - 1.) / 2.]
extent_indices = [int(center[0] + SubhaloPos[axes][1] - size), int(center[0] + SubhaloPos[axes][1] + size),
                  int(center[1] + SubhaloPos[axes][0] - size), int(center[1] + SubhaloPos[axes][0] + size)]
extent = [-size, size, -size, size]

# draw a square around the satellite, and include the zoom lines
ax1 = axs[0]
ax2 = axs[1]
xy = (SubhaloPos[axes][0] - size, SubhaloPos[axes][1] - size)
height = size*2
width = size*2

rect = patches.Rectangle(xy, width, height, fill=False)
ax.add_patch(rect)
con = ConnectionPatch(xyA=xy, coordsA=ax1.transData,
                      xyB=(0,0), coordsB=ax2.transAxes)
fig.add_artist(con)

xy2 = (xy[0], xy[1] + height)
con = ConnectionPatch(xyA=xy2, coordsA=ax1.transData,
                      xyB=(0,1), coordsB=ax2.transAxes)
fig.add_artist(con)

ax = axs[1]
img = ax.imshow(FoF_LXsoftexcess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]], extent=extent, cmap='GnBu', origin='lower',
                norm=mpl.colors.LogNorm(vmin=1.0e0, vmax=1.0e2))
cbar = fig.colorbar(img, ax=ax, extend='both', location='bottom')
cbar.set_label(LXsoftexcess_label, fontsize='small')
x0 = -size + (size*0.15)# kpc
y0 =  size - (size*0.1)# kpc
length = 1.0e2 # kpc
ax.plot([x0, x0+length], [y0, y0], color='black', marker='None', ls='-')
ax.text(x0+length/2., y0-(size * 0.1), '%d kpc'%(length), ha='center', va='top', fontsize='small', color='black')

ax.set_xticks([])
ax.set_yticks([])

ax = axs[2]
model, func = model_2dradprof(FoF_LXsoftexcess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]])
ax.plot(np.arange(size), func(np.arange(size)), '-')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'Satellite-centric Distance [kpc]')
ax.set_ylabel(LXsoftexcess_label)


In [ ]:
FoF_Lxsoftexcess = (FoF_Lxsoft - FoF_Lxsoft_model) / FoF_Lxsoft_model
plt.figure()
img = plt.imshow(FoF_Lxsoftexcess, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.], cmap='RdBu', origin='lower',
                 vmin=-1.0, vmax=1.0)
plt.colorbar(img)

In [ ]:
FoF_LXsoftexcess = (FoF_Lxsoft - FoF_Lxsoft_model)
plt.figure()
img = plt.imshow(FoF_LXsoftexcess, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.], cmap='RdBu', origin='lower',
                 norm=mpl.colors.SymLogNorm(1.0e4, vmin=-1.0e9, vmax=1.0e9))
plt.colorbar(img)

In [ ]:
subhaloPos = np.zeros((subhaloIDs.size, 3), dtype=float) 
satellite_HCDNz0 = np.zeros(subhaloIDs.size, dtype=float)
satellite_SHGMz0 = satellite_HCDNz0.copy()
for i, taudict_i in enumerate(satellite_taudict_indices):
    satellite_HCDNz0[i] = satellites_taudict[HCDnz0_key][taudict_i]
    satellite_SHGMz0[i] = satellites_taudict['SubhaloHotGasMass_z0'][taudict_i]
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloIDs[i])
    subhaloPos[i] = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)

subhaloPos_xy = subhaloPos[:,axes]
satellites_fov_indices = np.where((abs(subhaloPos_xy[:,0]) < R200c) & (abs(subhaloPos_xy[:,1]) < R200c))[0]
satellites_fov_indices.size

In [ ]:
start = np.argwhere(satellite_SHGMz0 == satellite_SHGMz0.max())[0][0]
end = start+1
for satellite_fov_index in satellites_fov_indices[start:start+5]:
    subhaloID = subhaloIDs[satellite_fov_index]
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloID)
    SubhaloPos = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)
    satellite_HCDNz0 = satellites_taudict[HCDnz0_key][satellite_taudict_indices[satellite_fov_index]]
    
    size = 3.0e2 # kpc

    center = [(FoF_LXsoftexcess.shape[0] - 1.) / 2., (FoF_LXsoftexcess.shape[1] - 1.) / 2.]
    extent_indices = [int(center[0] + SubhaloPos[axes][1] - size), int(center[0] + SubhaloPos[axes][1] + size),
                      int(center[1] + SubhaloPos[axes][0] - size), int(center[1] + SubhaloPos[axes][0] + size)]
    extent = [-size, size, -size, size]

    fig, axs = plt.subplots(1, 2, figsize=(figsizewidth, figsizewidth/3.))
    img = axs[0].imshow(FoF_LXsoftexcess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]], extent=extent, cmap='RdBu', origin='lower',
                        norm=mpl.colors.SymLogNorm(1.0e4, vmax=1.0e9))
    fig.colorbar(img, ax=axs[0])
    model, func = model_2dradprof(FoF_LXsoftexcess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]])
    axs[1].plot(np.arange(size), func(np.arange(size)), '-')
    axs[1].set_yscale('log')
    axs[1].set_xscale('log')
    

In [ ]:
satellite_index = 10
subhaloID = subhaloIDs[satellite_index]
subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloID)
SubhaloPos = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)
satellite_HCDNz0 = satellites_taudict[HCDnz0_key][satellite_taudict_indices[satellite_index]]
print(satellite_HCDNz0, SubhaloPos[axes], np.log10(satellites_taudict['SubhaloHotGasMass_z0'][satellite_taudict_indices[satellite_index]]))

In [ ]:
# calculate the gas cell sizes
Coordinates = Coordinates[:,axes]
quant = None
FoF_coldens = sphMap.sphMap(Coordinates, Sizes, Masses, quant, axes, boxSizeImg, boxSizeSim, boxCen, nPixels, ndims, colDens=True)

plt.figure()
img = plt.imshow(FoF_coldens, extent=[-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[1]/2., boxSizeImg[1]/2.], cmap='plasma', origin='lower',
                 norm=mpl.colors.LogNorm(vmin=1.0e3))


In [ ]:
# model the FoF gas cells as a 2D radial profile
FoF_coldens_model, FoF_coldens_model_func = model_2dradprof(FoF_coldens)

plt.figure()
img = plt.imshow(FoF_coldens_model, extent=[-boxSizeImg[0]/2., boxSizeImg[0]/2., -boxSizeImg[1]/2., boxSizeImg[1]/2.], cmap='plasma', origin='lower',
                 norm=mpl.colors.LogNorm(vmin=1.0e3))
plt.colorbar(img)

In [ ]:
FoF_excess = (FoF_coldens - FoF_coldens_model)
plt.figure()
img = plt.imshow(FoF_excess, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.], cmap='RdBu', origin='lower',
                 norm=mpl.colors.SymLogNorm(1.0e4, vmax=1.0e8))
plt.colorbar(img)

In [ ]:
subhaloPos = np.zeros((subhaloIDs.size, 3), dtype=float) 
satellite_HCDNz0 = np.zeros(subhaloIDs.size, dtype=float)
for i, taudict_i in enumerate(satellite_taudict_indices):
    satellite_HCDNz0[i] = satellites_taudict[HCDnz0_key][taudict_i]
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloIDs[i])
    subhaloPos[i] = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)

subhaloPos_xy = subhaloPos[:,axes]
satellites_fov_indices = np.where((abs(subhaloPos_xy[:,0]) < R200c) & (abs(subhaloPos_xy[:,1]) < R200c))[0]
satellites_fov_indices.size

In [ ]:
for satellite_fov_index in satellites_fov_indices[:5]:
    subhaloID = subhaloIDs[satellite_fov_index]
    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloID)
    SubhaloPos = ru.shift(subhalo['SubhaloPos'] * a / h, halo_pos, boxsize)
    satellite_HCDNz0 = satellites_taudict[HCDnz0_key][satellite_taudict_indices[satellite_fov_index]]
    
    size = 3.0e2 # kpc

    center = [(FoF_excess.shape[0] - 1.) / 2., (FoF_excess.shape[1] - 1.) / 2.]
    extent_indices = [int(center[0] + SubhaloPos[axes][1] - size), int(center[0] + SubhaloPos[axes][1] + size),
                      int(center[1] + SubhaloPos[axes][0] - size), int(center[1] + SubhaloPos[axes][0] + size)]
    extent = [-size, size, -size, size]

    fig, axs = plt.subplots(1, 2)
    img = axs[0].imshow(FoF_excess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]], extent=extent, cmap='ocean', origin='lower',
                        norm=mpl.colors.SymLogNorm(1.0e4, vmax=1.0e8))
    fig.colorbar(img, ax=axs[0])
    model, func = model_2dradprof(FoF_excess[extent_indices[0]:extent_indices[1],extent_indices[2]:extent_indices[3]])
    axs[1].plot(np.arange(size), func(np.arange(size)), '-')
    axs[1].set_yscale('log')
    

In [ ]:
# load a low resultion halo's gas cells
sim = 'L680n8192TNG'
basePath = ru.ret_basePath(sim)
snapNum = 99

# load general simulation parameters
header = il.groupcat.loadHeader(basePath, snapNum)
z = header['Redshift']
a = header['Time'] # scale factor
h = header['HubbleParam'] # = 0.6774
boxsize = header['BoxSize'] * a / h
    
gaspart_num = il.util.partTypeNum('gas')


axes = [0,1]
boxSizeImg = [4.0e3, 4.0e3, 6.0e3] # kpc
boxSizeSim = [boxsize, boxsize, boxsize]
boxCen = [0, 0, 0]
nPixels = [1600, 1600]
ndims = 3

In [ ]:
for i, haloID in enumerate(haloIDs[1:2]):
    subhaloID = central_subfindIDs[i]

    subhalo = il.groupcat.loadSingle(basePath, snapNum, subhaloID=subhaloID)
    subhalo_gas_cells = il.snapshot.loadSubhalo(basePath, snapNum, subhaloID, 'gas')

    Pos = subhalo['SubhaloPos'] * a / h

    Masses = subhalo_gas_cells['Masses'] * 1.0e10 / h
    Coordinates = subhalo_gas_cells['Coordinates'] * a / h
    Coordinates = ru.shift(Coordinates, Pos, boxsize)
    Densities = subhalo_gas_cells['Density'] * 1.0e10 / h / (a / h)**3
    Sizes = (Masses / (Densities * 4./3. * np.pi))**(1./3.)

    quant = None
    subhalo_gascoldens = sphMap.sphMap(Coordinates, Sizes, Masses, quant, axes, boxSizeImg, boxSizeSim, boxCen, nPixels, ndims, colDens=True)

    plt.figure()
    img = plt.imshow(subhalo_gascoldens, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.], cmap='plasma',
                     norm=mpl.colors.LogNorm(vmin=1.0e3))
    cbar = plt.colorbar(img)
    cbar.set_label(r'Gas Mass Column Density [${\rm M_\odot\, kpc^{-2}}$]')
    plt.xlabel(r'$x$ [kpc]')
    plt.ylabel(r'$y$ [kpc]')
    plt.title(r'TNG-Cluster subhaloID %d'%subhaloID)
    outdirec = '../Figures/TNGCluster/Maps/'
    fname = 'TNGCluster_haloID-%08d_SubhaloGasMassMap.pdf'%haloID
    #plt.savefig(outdirec + fname, bbox_inches='tight')

    subhalo_model = model_2dradprof(subhalo_gascoldens)

    plt.figure()
    plt.imshow(subhalo_model, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.], cmap='plasma',
                     norm=mpl.colors.LogNorm(vmin=1.0e3))
    cbar = plt.colorbar(img)
    cbar.set_label(r'Gas Mass Column Density [${\rm M_\odot\, kpc^{-2}}$]')
    plt.xlabel(r'$x$ [kpc]')
    plt.ylabel(r'$y$ [kpc]')
    plt.title(r'TNG-Cluster subhaloID %d model'%subhaloID)
    fname = 'TNGCluster_haloID-%08d_SubhaloGasMassModel.pdf'%haloID
    #plt.savefig(outdirec + fname, bbox_inches='tight')    
    
    subhalo_excess = (subhalo_gascoldens - subhalo_model) / subhalo_model
    
    plt.figure()
    img = plt.imshow(subhalo_excess, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.], cmap='RdBu', vmin=-1.0, vmax=1.0)
    cbar = plt.colorbar(img)
    cbar.set_label(r'Gas Mass Column Density Excess')
    plt.xlabel(r'$x$ [kpc]')
    plt.ylabel(r'$y$ [kpc]')
    plt.title(r'TNG-Cluster subhaloID %d'%subhaloID)
    fname = 'TNGCluster_haloID-%08d_SubhaloGasMassExcess.pdf'%haloID
    #plt.savefig(outdirec + fname, bbox_inches='tight')

    halo = il.groupcat.loadSingle(basePath, snapNum, haloID=haloID)
    halo_gas_cells = il.snapshot.loadHalo(basePath, snapNum, haloID, 'gas')

    Masses = halo_gas_cells['Masses'] * 1.0e10 / h
    Coordinates = halo_gas_cells['Coordinates'] * a / h
    Coordinates = ru.shift(Coordinates, Pos, boxsize)
    Densities = halo_gas_cells['Density'] * 1.0e10 / h / (a / h)**3
    Sizes = (Masses / (Densities * 4./3. * np.pi))**(1./3.)
    Temperature = ru.calc_temp_dict(halo_gas_cells)['Temperature']
    GFM_Metallicity = halo_gas_cells['GFM_Metallicity'] / 0.0127

    quant = None
    halo_gascoldens = sphMap.sphMap(Coordinates, Sizes, Masses, quant, axes, boxSizeImg, boxSizeSim, boxCen, nPixels, ndims, colDens=True, minIntProj=False)

    plt.figure()
    img = plt.imshow(halo_gascoldens, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.], cmap='plasma',
                     norm=mpl.colors.LogNorm(1.0e3))
    cbar = plt.colorbar(img)
    cbar.set_label(r'Gas Mass Column Density [${\rm M_\odot\, kpc^{-2}}$]')
    plt.xlabel(r'$x$ [kpc]')
    plt.ylabel(r'$y$ [kpc]')
    plt.title(r'TNG-Cluster halo %d'%haloID)
    fname = 'TNGCluster_haloID-%08d_GasMassMap.pdf'%haloID
    #plt.savefig(outdirec + fname, bbox_inches='tight')

    halo_excess = (halo_gascoldens - subhalo_model) / subhalo_model

    plt.figure()
    img = plt.imshow(halo_excess, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.], cmap='RdBu', vmin=-1.0, vmax=1.0)
    cbar = plt.colorbar(img)
    cbar.set_label(r'Gas Mass Column Density Excess')
    plt.xlabel(r'$x$ [kpc]')
    plt.ylabel(r'$y$ [kpc]')
    plt.title(r'TNG-Cluster haloID %d'%haloID)
    fname = 'TNGCluster_haloID-%08d_GasMassExcess.pdf'%haloID
    #plt.savefig(outdirec + fname, bbox_inches='tight')
    
    
    quant = np.log10(Temperature)
    halo_gastemp = sphMap.sphMap(Coordinates, Sizes, Masses, quant, axes, boxSizeImg, boxSizeSim, boxCen, nPixels, ndims, colDens=False)

    plt.figure()
    img = plt.imshow(halo_gastemp, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.],
                     cmap='rainbow', vmin=7.0, vmax=8.0)
    cbar = plt.colorbar(img)
    cbar.set_label(r'Gas Temperature [log K]')
    plt.xlabel(r'$x$ [kpc]')
    plt.ylabel(r'$y$ [kpc]')
    plt.title(r'TNG-Cluster haloID %d'%haloID)
    outdirec = '../Figures/TNGCluster/Maps/'
    fname = 'TNGCluster_haloID-%08d_GasTempMap.pdf'%haloID
    #plt.savefig(outdirec + fname, bbox_inches='tight')
    
    
    quant = np.log10(GFM_Metallicity)
    halo_gasmetallicity = sphMap.sphMap(Coordinates, Sizes, Masses, quant, axes, boxSizeImg, boxSizeSim, boxCen, nPixels, ndims, colDens=False)

    plt.figure()
    img = plt.imshow(halo_gasmetallicity, extent=[-boxSizeImg[axes[0]]/2., boxSizeImg[axes[0]]/2., -boxSizeImg[axes[1]]/2., boxSizeImg[axes[1]]/2.],
                     cmap='viridis', vmin=-1.0, vmax=-0.5)
    cbar = plt.colorbar(img)
    cbar.set_label(r'Gas Metallicity [log Z$_{\odot}$]')
    plt.xlabel(r'$x$ [kpc]')
    plt.ylabel(r'$y$ [kpc]')
    plt.title(r'TNG-Cluster haloID %d'%haloID)
    outdirec = '../Figures/TNGCluster/Maps/'
    fname = 'TNGCluster_haloID-%08d_GasMetallicityMap.pdf'%haloID
    #plt.savefig(outdirec + fname, bbox_inches='tight')
    
    #plt.close('all')
    
    


In [ ]:
halo_gascoldens.max()